In [1]:
%env CUDA_VISIBLE_DEVICES=0
from utils import *
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from region_traverser import *

model_name = "showlab/ShowUI-2B"
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation='flash_attention_2'
)
min_pixels = 256*28*28
max_pixels = 1344*28*28

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

ds = load_screenspot_ds()

env: CUDA_VISIBLE_DEVICES=0


2024-12-19 22:37:04.444702: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-19 22:37:04.445996: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-19 22:37:04.469497: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-19 22:37:05.045902: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other 

[2024-12-19 22:37:06,248] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cpu (auto detect)


/home/trainer/anaconda3/envs/ml/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [2]:
def get_coordinate_prediction(image, target):
    #img_url = 'examples/web_dbd7514b-9ca3-40cd-b09a-990f7b955da1.png'
    query = target
    
    
    _SYSTEM = "Based on the screenshot of the page, I give a text description and you give its corresponding location. The coordinate represents a clickable location [x, y] for an element, which is a relative coordinate on the screenshot, scaled from 0 to 1."
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": _SYSTEM},
                {"type": "image", "image": image},
                {"type": "text", "text": query}
            ],
        }
    ]
    
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True,
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")
    
    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]
    
    x, y = tuple(ast.literal_eval(output_text))
    
    result = (x*999, y*999)
    return result

In [8]:
import ast
dim = 999

def eval_row_baseline(row, verbose=False):
    target = row['target']
    og_size = row['image'].size
    current_region = row['image'].resize((dim,dim))
    x, y = get_coordinate_prediction([current_region], target)
    x = x * current_region.size[0]
    x = y * current_region.size[1]
    porp_x = x / dim
    porp_y = y / dim

    x = og_size[0] * porp_x
    y = og_size[1] * porp_y

    if verbose:
        print(target)
        print(current_region.size)
        print(x,y)
        render_crosshair(row['image'], x, y).convert("RGB").show()
    return is_in_bbox(row['bbox'], x, y)

def eval_row(row, verbose=False):
    target = row['target']
    current_region = row['image'].copy()

    traverser = RegionTraverser(current_region)
    if verbose: print(target)
    k = 3
    #render_crosshair_center(current_region).show()
    for i in range(k):
        image_prompt = current_region.resize((dim, dim))
        prediction_coord = get_coordinate_prediction(image_prompt, target)
        pred_x, pred_y = prediction_coord
        
        if verbose: print(prediction_coord)
        if i != k-1:
            traverser.consume_coordinate(pred_x, pred_y)
            result_image = traverser.get_highlighted_image()
            current_region = traverser.get_cropped_image().resize((dim,dim))

        if verbose: result_image.convert("RGB").show()
    final_bbox = traverser.get_bounding_box()

    last_porp_x = pred_x / dim
    last_porp_y = pred_y / dim
    
    delta_x = (final_bbox[2] - final_bbox[0]) * last_porp_x
    delta_y = (final_bbox[3] - final_bbox[1]) * last_porp_y

    x,y = final_bbox[0] + delta_x, final_bbox[1] + delta_y
    if verbose:
        render_crosshair(current_region, pred_x, pred_y).convert("RGB").show()
        render_crosshair(row['image'], x, y).convert("RGB").show()
        #draw_bbox_on_image(row['image'], pred_bbox).convert("RGB").show()
    return is_in_bbox(row['bbox'], x, y)

#eval_row(ds['web']['text'][0], True)

In [5]:
from tqdm import tqdm
devices = ["web", "mobile", "desktop"]
ui_types = ["text", "icon"]

weights = {}
eval_result = {}

for d in devices:
    for t in ui_types:
        test_set = ds[d][t]
        weights[f"{d} - {t}"] = len(test_set)
        results = []
        for row in tqdm(test_set):
            res = eval_row(row)
            results.append(res)
        eval_result[f"{d} - {t}"] = len([x for x in results if x]) / len(test_set)

100%|█████████████████████████████████████████████████████████████████████████████████| 140/140 [03:56<00:00,  1.69s/it]


In [6]:
eval_result

{'web - text': 0.8304347826086956,
 'web - icon': 0.6893203883495146,
 'mobile - text': 0.8901098901098901,
 'mobile - icon': 0.74235807860262,
 'desktop - text': 0.8762886597938144,
 'desktop - icon': 0.6857142857142857}

In [7]:
import numpy as np
np.average(list(eval_result.values()), weights=list(weights.values()))

0.7955974842767296